***1. dispy can be used as a command line tool (for simple cases, scheduling cron jobs); in this case the computations should only be programs and dependencies should only be files.***

In [ ]:
dispy_cmd.py -f /some/file1 -f file2 -a "arg11 arg12" -a "arg21 arg22" -a "arg3" /some/program

**2. A simple client program that distributes a program (say, ‘/path/to/program’), executes them with a sequence of numbers as arguments is:**

In [ ]:
import dispy
cluster = dispy.JobCluster('/path/to/program')
for i in range(50):
    cluster.submit(i)

The program ‘/path/to/program’ on the client computer is transferred to each of the nodes, so if the program is a binary program then all the nodes should have same architecture as the client.

***3. Create a program. A canonical cluster that distributes computation ‘compute’ (Python function) to nodes (running dispynode (server) on a local network), schedules jobs with the cluster, gets jobs’ results and prints them is***

In [ ]:
# function 'compute' is distributed and executed with arguments
# supplied with 'cluster.submit' below
def compute(n):
    import time, socket
    time.sleep(n)
    host = socket.gethostname()
    return (host, n)

if __name__ == '__main__':
    # executed on client only; variables created below, including modules imported,
    # are not available in job computations
    import dispy, random
    # distribute 'compute' to nodes; in this case, 'compute' does not have
    # any dependencies to run on nodes
    cluster = dispy.JobCluster(compute)
    # run 'compute' with 20 random numbers on available CPUs
    jobs = []
    for i in range(20):
        job = cluster.submit(random.randint(5,20))
        jobs.append(job)
    # cluster.wait() # waits until all jobs finish
    for job in jobs:
        host, n = job() # waits for job to finish and returns results
        print('%s executed job %s at %s with %s' % (host, job.id, job.start_time, n))
        # other fields of 'job' that may be useful:
        # job.stdout, job.stderr, job.exception, job.ip_addr, job.end_time
    cluster.print_status()  # shows which nodes executed how many jobs etc.

***4. write a program distributes computation to be executed with instances of a class***

In [ ]:
class C(object):
    def __init__(self, i, n):
        self.i = i
        self.n = n

    def show(self):
        print('%s: %.2f' % (self.i, self.n))

def compute(obj):
    # obj is an instance of C
    import time
    time.sleep(obj.n)
    obj.show()  # the output is stored in job.stdout
    return obj.n

if __name__ == '__main__':
    import random, dispy
    # 'compute' needs definition of class C, so include in 'depends'
    cluster = dispy.JobCluster(compute, depends=[C])
    jobs = []
    for i in range(10):
        c = C(i, random.uniform(1, 3)) # create object of C
        job = cluster.submit(c) # it is sent to a node for executing 'compute'
        job.id = c # store this object for later use
        jobs.append(job)
    for job in jobs:
        job() # wait for job to finish
        print('%s: %.2f / %s' % (job.id.i, job.result, job.stdout))